In [ ]:
# Standard library
from collections import OrderedDict
import os

# Third-party
import astropy.time as atime
from astropy import log as logger
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline
import numpy as np
import corner

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.units import usys
from thejoker.celestialmechanics import OrbitalParams, SimulatedRVOrbit, rv_from_elements
from thejoker.plot import plot_rv_curves, plot_corner

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
data_filename = "../data/experiment5.h5"

plot_path = "../paper/figures/"
os.makedirs(plot_path, exist_ok=True)

## Read the data

In [ ]:
all_data = OrderedDict()
with h5py.File(data_filename, 'r') as f:
    for key in sorted([int(x) for x in f.keys()]):
        all_data[key] = RVData.from_hdf5(f[str(key)])

In [ ]:
dmjd = all_data[0].t.tcb.mjd.max() - all_data[0].t.tcb.mjd.min()
t_grid = np.linspace(all_data[0].t.tcb.mjd.min() -0.25*dmjd, 
                     all_data[0].t.tcb.mjd.max() + 0.25*dmjd, 
                     1024)

rv_unit = u.km/u.s

In [ ]:
fig,axes = plt.subplots(2, 1, figsize=(5,10), 
                        sharex=True, sharey=True)   

for i,key in enumerate(list(all_data.keys())):
    data = all_data[key]
    data.plot(ax=axes[i], rv_unit=u.km/u.s, markersize=3, capsize=0,
              elinewidth=0, color='k', zorder=1000, alpha=0.75)
    
    # read the orbital parameters
#     samples_filename = "../cache/experiment5-{}.h5".format(key)
#     with h5py.File(samples_filename, 'r') as g:            
#         opars = OrbitalParams.from_hdf5(g)
#         samples = opars.pack(plot_units=True)
#         print(samples.shape)

#     plot_rv_curves(opars[:128], t_grid, t_offset=data.t_offset, rv_unit=rv_unit, 
#                    ax=axes[i], plot_kwargs={'color': '#888888', 'zorder': -100},
#                    add_labels=False)
#     axes[i].set_rasterization_zorder(-1)

axes[0].set_ylim(0, 30)

fig.tight_layout()

### Two-column, rv curves + period-eccentricity

In [ ]:
fig = plt.figure(figsize=(10,6.5))
gs = gridspec.GridSpec(2, 3)

axes_l = []
axes_r = []
for i,key in enumerate(all_data.keys()):
    if len(axes_l) == 0:
        ax_l = fig.add_subplot(gs[i, :2])
        ax_r = fig.add_subplot(gs[i, 2])
    else:
        ax_l = fig.add_subplot(gs[i, :2], sharex=axes_l[0], sharey=axes_l[0])
        ax_r = fig.add_subplot(gs[i, 2], sharex=axes_r[0], sharey=axes_r[0])
    axes_l.append(ax_l)
    axes_r.append(ax_r)

    data = all_data[key]

    # read the orbital parameters
    samples_filename = "../cache/experiment5-{}.h5".format(key)
    with h5py.File(samples_filename, 'r') as g:            
        opars = OrbitalParams.from_hdf5(g)
        samples = opars.pack(plot_units=True)
        print(samples.shape)

    plot_rv_curves(opars[:128], t_grid, t_offset=data.t_offset, rv_unit=rv_unit, 
                   ax=ax_l, plot_kwargs={'color': '#888888', 'zorder': -100},
                   add_labels=False)
    ax_l.set_rasterization_zorder(-1)
    data.plot(ax=ax_l, rv_unit=u.km/u.s, markersize=3, capsize=0,
              elinewidth=0, color='k', zorder=1000, alpha=0.75)
#     ax_l.text(55650, -9, "$N={}$, M={}".format(len(data), len(joker_opars)), va='top', fontsize=18)

    # plot markers on right panel
    _n = len(opars)
    y1,y2 = 0.75, 0.05
    alpha = (y2-y1) * (np.log(_n) - np.log(37)) / (np.log(461789)-np.log(37)) + y1

    y1,y2 = 4,1
    size = max(1, int((y2-y1) * (np.log(_n) - np.log(37)) / (np.log(10000)-np.log(37)) + y1))

    style = dict(alpha=alpha, marker='.', markersize=size, linestyle='none', 
                 rasterized=True, color='#888888')
    ax_r.plot(samples[:,0], samples[:,2], **style)

    if i == 0:
        print("hide", i)
        plt.setp(ax_l.get_xticklabels(), visible=False)
        plt.setp(ax_r.get_xticklabels(), visible=False)

    ax_l.set_ylabel('RV [km s$^{-1}$]')
    ax_r.set_ylabel(OrbitalParams._latex_labels[2])

axes_l[-1].set_xlabel('BMJD')
axes_r[-1].set_xlabel(OrbitalParams._latex_labels[0])

axes_l[0].set_xlim(t_grid.min(), t_grid.max())
axes_l[0].set_ylim(0, 30)

# axes_r[0].set_xlim(ln_P_lims[j])
# axes_r[0].set_ylim(ecc_lims[j])

# hack:
# if j == 0:
#     axes_r[0].yaxis.set_ticks([0, 0.1, 0.2])
#     axes_r[0].xaxis.set_ticks([5.1,5.2,5.3,5.4])

fig.tight_layout()

# change to: dpi=256 for production?
# fig.savefig(os.path.join(plot_path, 'exp5-rv-curves-{}.pdf'.format(j)), dpi=100) 

In [ ]:
samples_filename = "../cache/experiment5-{}.h5".format(0)
with h5py.File(samples_filename, 'r') as g:            
    opars = OrbitalParams.from_hdf5(g)
    samples = opars.pack(plot_units=True)

In [ ]:
plt.plot(samples[:,0], samples[:,1])

--- 

### Old version:

In [ ]:
n_panel_per_fig = 4

for j in range(len(all_data) // n_panel_per_fig):
    fig,axes = plt.subplots(n_panel_per_fig, 1, figsize=(6.5,10), 
                            sharex=True, sharey=True)   

    for i,key in enumerate(list(all_data.keys())[j*n_panel_per_fig:(j+1)*n_panel_per_fig]):
        data = all_data[key]
        
        # read the orbital parameters
        samples_filename = "../cache/experiment3-{}.h5".format(key)
        with h5py.File(samples_filename, 'r') as g:            
            if 'emcee' in g:
                opars = OrbitalParams.from_hdf5(g['emcee'])
            else:
                opars = OrbitalParams.from_hdf5(g)
        
        plot_rv_curves(opars[:128], t_grid, t_offset=data.t_offset, rv_unit=rv_unit, 
                       ax=axes[i], plot_kwargs={'color': '#888888', 'zorder': -100},
                       add_labels=False)
        axes[i].set_rasterization_zorder(-1)
        
        data.plot(ax=axes[i], rv_unit=u.km/u.s, markersize=3, capsize=0,
                  elinewidth=0, color='k', zorder=1000, alpha=0.75)
        
        if key > 0:
            not_included = np.array([x not in data._rv for x in all_data[0]._rv])
            reject_data = all_data[0][not_included]
            
            axes[i].plot(reject_data.t.tcb.mjd, reject_data.rv.to(u.km/u.s).value, 
                         marker='x', zorder=100, linestyle='none',
                         markersize=4, markeredgewidth=1, color='#de2d26')
        
        axes[i].text(55650, -9, "$N={}$, M={}".format(len(data), len(opars)), va='top', fontsize=18)
    
    axes[0].set_ylabel('RV [km s$^{-1}$]')
    axes[-1].set_xlabel('BMJD')
    
    axes[0].set_xlim(t_grid.min(), t_grid.max())
    axes[0].set_ylim(-55, -5)

    fig.tight_layout()
    
    # change to: dpi=256 for production?
#     fig.savefig(os.path.join(plot_path, 'exp2-rv-curves-{}.pdf'.format(j)), dpi=100) 

## Corner plots

In [ ]:
for key in all_data.keys():
    data = all_data[key]

    # read the orbital parameters
    samples_filename = "../cache/experiment2-{}.h5".format(key)
    with h5py.File(samples_filename, 'r') as g:
        opars = OrbitalParams.from_hdf5(g)
    
    _n = len(opars)
        
    y1,y2 = 3,1
    size = int((y2-y1) * (np.log(_n) - np.log(37)) / (np.log(461789)-np.log(37)) + y1)
    
    print(_n, size)

In [ ]:
n_panel_per_fig = 4

for j in range(len(all_data) // n_panel_per_fig):
    fig,axes = plt.subplots(n_panel_per_fig, 3, figsize=(8,11), 
                            sharex='col', sharey=True)   

    for i,key in enumerate(list(all_data.keys())[j*n_panel_per_fig:(j+1)*n_panel_per_fig]):
        data = all_data[key]
        
        # read the orbital parameters
        samples_filename = "../cache/experiment2-{}.h5".format(key)
        with h5py.File(samples_filename, 'r') as g:
            opars = OrbitalParams.from_hdf5(g)
        
        samples = opars.pack(plot_units=True)
        
#         kw = dict(plot_contours=False, plot_density=False, 
#                   data_kwargs=dict(alpha=0.05, marker=','))
#         corner.hist2d(samples[:,1], samples[:,0], ax=axes[i,0], **kw)
#         corner.hist2d(samples[:,2], samples[:,0], ax=axes[i,1], **kw)
#         corner.hist2d(samples[:,5], samples[:,0], ax=axes[i,2], **kw)
        
        _n = len(opars)
        y1,y2 = 0.75, 0.05
        alpha = (y2-y1) * (np.log(_n) - np.log(37)) / (np.log(461789)-np.log(37)) + y1
        
        y1,y2 = 4,1
        size = max(1, int((y2-y1) * (np.log(_n) - np.log(37)) / (np.log(10000)-np.log(37)) + y1))
    
        print(_n, alpha, size)
    
        style = dict(alpha=alpha, marker='.', markersize=size, linestyle='none', rasterized=True)
        axes[i,0].plot(samples[:,1], samples[:,0], **style)
        axes[i,1].plot(samples[:,2], samples[:,0], **style)
        axes[i,2].plot(samples[:,5], samples[:,0], **style)
        
        axes[i,0].set_ylabel(opars._latex_labels[0])
        
    axes[0,0].set_xlim(-0.5, 7.5) # ln asini
    axes[0,1].set_xlim(-0.1, 1.1) # ecc
    axes[0,2].set_xlim(-75, 15) # v0
    
    axes[0,0].set_ylim(2.5, 7.5) # ln P
    
    axes[-1,0].set_xlabel(opars._latex_labels[1])
    axes[-1,1].set_xlabel(opars._latex_labels[2])
    axes[-1,2].set_xlabel(opars._latex_labels[5])
    
    fig.tight_layout()
    
    # change to: dpi=256 for production?
    fig.savefig(os.path.join(plot_path, 'exp2-corner-{}.pdf'.format(j)), dpi=100) 